In [87]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os

from functools import reduce
from dotenv import load_dotenv
import plotly.graph_objects as go

import gmplot

In [21]:
resp = requests.get("https://datos.cdmx.gob.mx//api/records/1.0/search/?dataset=carpetas-de-investigacion-pgj-de-la-ciudad-de-mexico&rows=10000&facet=ao_hechos&facet=mes_hechos&facet=delito&facet=categoria_delito&facet=fiscalia&facet=alcaldia_hechos&facet=ao_inicio")

In [26]:
resp.json()

{'nhits': 851010,
 'parameters': {'dataset': 'carpetas-de-investigacion-pgj-de-la-ciudad-de-mexico',
  'timezone': 'UTC',
  'rows': 10000,
  'format': 'json',
  'facet': ['ao_hechos',
   'mes_hechos',
   'delito',
   'categoria_delito',
   'fiscalia',
   'alcaldia_hechos',
   'ao_inicio']},
 'records': [{'datasetid': 'carpetas-de-investigacion-pgj-de-la-ciudad-de-mexico',
   'recordid': '9a29b80ee52814fe546e2609d7387b034466924c',
   'fields': {'latitud': '19.2584328792475',
    'mes_hechos': 'Marzo',
    'ao_inicio': '2018',
    'fecha_inicio': '2018-04-01 00:03',
    'calle_hechos': 'COLIBRI',
    'agencia': 'XO-2',
    'alcaldia_hechos': 'XOCHIMILCO',
    'categoria_delito': 'DELITO DE BAJO IMPACTO',
    'fiscalia': 'INVESTIGACIÓN EN XOCHIMILCO',
    'unidad_investigacion': 'UI-1SD',
    'fecha_hechos': '2018-03-31 18:30',
    'colonia_hechos': 'CALTONGO',
    'geopoint': [19.2584328792, -99.0846002377],
    'ao_hechos': '2018',
    'delito': 'VIOLENCIA FAMILIAR',
    'mes_inicio': '

In [38]:
df_list = []
for i in range(10000):
    df_list.append(pd.Series(pd.DataFrame(resp.json()["records"]).fields.iloc[i]))

In [46]:
df = pd.DataFrame(df_list)

In [66]:
df.head()

,latitud,mes_hechos,ao_inicio,fecha_inicio,calle_hechos,agencia,alcaldia_hechos,categoria_delito,fiscalia,unidad_investigacion,fecha_hechos,colonia_hechos,geopoint,ao_hechos,delito,mes_inicio,longitud,calle_hechos2
0,19.258433,Marzo,2018,2018-04-01 00:03:00,COLIBRI,XO-2,XOCHIMILCO,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN XOCHIMILCO,UI-1SD,2018-03-31 18:30:00,CALTONGO,"[19.2584328792, -99.0846002377]",2018,VIOLENCIA FAMILIAR,Abril,-99.084600,NaN
1,NaN,Marzo,2018,2018-04-01 00:36:00,AVENIDA LA TURBA,TLH-1,TLAHUAC,HECHO NO DELICTIVO,INVESTIGACIÓN EN TLAHUAC,UI-1SD,2018-03-31 12:40:00,NaN,NaN,2018,PERDIDA DE LA VIDA POR PARO CARDIACO,Abril,NaN,NaN
2,19.304978,Abril,2018,2018-04-01 00:50:00,PERIFERICO SUR,TLP-4,TLALPAN,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN TLALPAN,UI-1CD,2018-04-01 00:00:00,JARDINES EN LA MONTAÑA,"[19.3049776399, -99.2041597379]",2018,NARCOMENUDEO POSESION SIMPLE,Abril,-99.204160,NaN
3,19.329496,Marzo,2018,2018-04-01 00:53:00,ALTA TENSION,AO-3,ALVARO OBREGON,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN ÁLVARO OBREGÓN,UI-1CD,2018-03-31 23:50:00,TORRES DE POTRERO,"[19.3294958332, -99.2554776348]",2018,CONTRA FUNCIONARIOS PUBLICOS,Abril,-99.255478,FLOTS
4,19.500777,Abril,2018,2018-04-01 03:23:00,POLITECNICO,GAM-2,GUSTAVO A MADERO,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN GUSTAVO A. MADERO,UI-1CD,2018-04-01 01:10:00,NUEVA INDUSTRIAL VALLEJO,"[19.5007766467, -99.148973921]",2018,LESIONES CULPOSAS POR TRANSITO VEHICULAR EN CO...,Abril,-99.148974,Y AVENIDA DE LOS 100 METROS


In [59]:
df.latitud = df.latitud.apply(lambda x: float(x) if x != "NA" else np.nan)
df.longitud = df.longitud.apply(lambda x: float(x) if x != "NA" else np.nan)

In [64]:
df.fecha_hechos = pd.to_datetime(df.fecha_hechos)
df.fecha_inicio = pd.to_datetime(df.fecha_inicio)

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
latitud                 9661 non-null float64
mes_hechos              10000 non-null object
ao_inicio               10000 non-null object
fecha_inicio            10000 non-null datetime64[ns]
calle_hechos            9949 non-null object
agencia                 10000 non-null object
alcaldia_hechos         9914 non-null object
categoria_delito        10000 non-null object
fiscalia                10000 non-null object
unidad_investigacion    10000 non-null object
fecha_hechos            10000 non-null datetime64[ns]
colonia_hechos          9658 non-null object
geopoint                9661 non-null object
ao_hechos               10000 non-null object
delito                  10000 non-null object
mes_inicio              10000 non-null object
longitud                9661 non-null float64
calle_hechos2           3891 non-null object
dtypes: datetime64[ns](2), float64(2), object(14)
mem

In [82]:
load_dotenv("")
mapbox_token = os.getenv("mapbox_token")

In [113]:
fig = go.Figure(go.Scattermapbox(lat=df.latitud,
                                 lon=df.longitud,
                                 mode='markers',
                                 marker=go.scattermapbox.Marker(
                                     size=9,
                                     opacity=0.2
                                 )
                                ))

fig.update_layout(autosize=True,
                  hovermode='closest',
                  mapbox=go.layout.Mapbox(accesstoken=mapbox_token,
                                          bearing=0,
                                          center=go.layout.mapbox.Center(
                                              lat=19.4,
                                              lon=-99.15
                                          ),
                                          pitch=0,
                                          zoom=10
                                         ),
                  width=800, 
                  height=800
                 )

fig.show()